In [52]:
import pandas as pd
import numpy as np
import json
import numpy as np
import gc
import warnings
warnings.filterwarnings('ignore')
from impala.dbapi import connect
from impala.util import as_pandas
import pymysql
import ast
import requests
import random

In [53]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [54]:
class global_var:
    """ 全局参数配置 """
    # HIVE地址
    game_id = 1001703
    media_id = 10
    platform = "['ANDROID']"
    budget = 3300
    ad_account_id_group = np.array([11596, 11597, 11598, 11599, 11600])
#     ad_account_id_group = np.array([6866, 6981, 6989, 6990, 6987, 10781])
    plan_num = 3
    

### 选优质素材

In [55]:
def get_game_id():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL 
    '''
    cur.execute(sql)
    result_df = cur.fetchall()
    cur.close()
    conn.close()
    return result_df

In [56]:
# 获取近期所有计划('2021年4月23号开始')
def get_plan_info():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
        SELECT
                * 
            FROM
                db_ptom.ptom_third_plan p
            WHERE
                game_id IN ({})
                AND media_id = 10
                AND create_time>=date( NOW() - INTERVAL 1440 HOUR )
                AND create_time<= date(NOW())
                            AND plan_id >= (
                                select plan_id from db_ptom.ptom_plan
                                where create_time >= date( NOW() - INTERVAL 1440 HOUR )
                                and create_time <= date( NOW() - INTERVAL 1416 HOUR )
                                limit 1
                            )
    '''
    finalSql = sql.format(game_id)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    return result_df

# 解析json
def get_plan_json(plan_info):
    plan_info.drop(['inventory_type', 'budget'], axis=1, inplace=True)
    plan_info.dropna(how='all', inplace=True, axis=1)
    plan_info.dropna(subset=['ad_info'], inplace=True)
    # 解析json
    plan_info['ad_info'] = plan_info['ad_info'].apply(json.loads)
    temp = plan_info['ad_info'].apply(pd.Series)
    plan_info = pd.concat([plan_info, temp], axis=1)
    plan_info.drop('ad_info', axis=1, inplace=True)
    temp = plan_info['audience'].apply(pd.Series)
    plan_info = pd.concat([plan_info, temp], axis=1)
    plan_info.drop('audience', axis=1, inplace=True)
    temp = plan_info['action'].apply(pd.Series)
    plan_info = pd.concat([plan_info, temp], axis=1)
    plan_info.drop('action', axis=1, inplace=True)
    plan_info.dropna(how='all', inplace=True, axis=1)

    for col in ['ad_account_id', 'game_id', 'channel_id', 'source_id',
                           'create_time', 'smart_bid_type', 'hide_if_exists', 'budget',
                           'delivery_range', 'adjust_cpa', 'inventory_type', 'hide_if_converted',
                           'flow_control_mode', 'schedule_time', 'cpa_bid', 'auto_extend_enabled',
                           'gender', 'city', 'platform', 'launch_price',
                           'retargeting_tags_exclude', 'interest_categories',
                           'ac', 'android_osv', 'location_type', 'retargeting_tags_include',
                           'ios_osv', 'interest_action_mode', 'age',
                           'action_categories', 'action_days', 'action_scene', 'deep_bid_type', 'roi_goal']:
        if col in plan_info.columns:
            pass
        else:
            plan_info[col] = np.nan
    plan_info = plan_info[['ad_account_id', 'game_id', 'channel_id', 'source_id',
                           'create_time', 'smart_bid_type', 'hide_if_exists', 'budget',
                           'delivery_range', 'adjust_cpa', 'inventory_type', 'hide_if_converted',
                           'flow_control_mode', 'schedule_time', 'cpa_bid', 'auto_extend_enabled',
                           'gender', 'city', 'platform', 'launch_price',
                           'retargeting_tags_exclude', 'interest_categories',
                           'ac', 'android_osv', 'location_type', 'retargeting_tags_include',
                           'ios_osv', 'interest_action_mode', 'age',
                           'action_categories', 'action_days', 'action_scene', 'deep_bid_type', 'roi_goal']]
    return plan_info

def get_now_plan_roi():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            a.ad_account_id,
            b.channel_id,
            b.source_id,
            b.tdate,
            b.amount,
            b.new_role_money,
            b.new_role_money / b.amount AS roi,
            b.pay_role_user_num / b.create_role_num AS pay_rate
        FROM
            db_data_ptom.ptom_plan a
        left join
            db_stdata.st_lauch_report b
        on a.chl_user_id=b.channel_id and a.source_id=b.source_id
        WHERE
            b.tdate >= date( NOW() - INTERVAL 240 HOUR )
            AND b.tdate_type = 'day'
            AND b.media_id = 10
            AND b.game_id IN ({})
            AND b.amount >= 200
            AND b.pay_role_user_num >= 1
            AND b.new_role_money >= 12
            AND (b.new_role_money / b.amount)>=0.005
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    result_df['tdate'] = pd.to_datetime(result_df['tdate'])
    result_df = result_df.sort_values('tdate')
    result_df = result_df.drop_duplicates(['channel_id', 'source_id'], keep='first')
    #     result_df = result_df[result_df['roi'] >= 0.03]
    return result_df

# 获取近期优化计划的创意数据
def get_creative():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/ 
        SELECT
            b.chl_user_id AS channel_id,
            b.source_id,
            JSON_EXTRACT( a.creative_param, '$.ad_keywords' ) AS ad_keywords,
            JSON_EXTRACT( a.creative_param, '$.title_list' ) AS title_list,
            JSON_EXTRACT( a.creative_param, '$.third_industry_id' ) AS third_industry_id 
        FROM
            db_ptom.ptom_batch_ad_task a
            LEFT JOIN db_ptom.ptom_plan b ON a.plan_name = b.plan_name 
        WHERE
            a.media_id = 10 
            AND b.create_time >= date( NOW() - INTERVAL 720 HOUR )    # 近30天
            AND a.game_id IN ({}) 
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    return result



In [57]:
def get_info():
    plan_info = get_plan_info()
    plan_info = get_plan_json(plan_info)
    creative_info = get_creative()
    creative_info['title_list'] = creative_info['title_list'].fillna('[]')
    creative_info['ad_keywords'] = creative_info['ad_keywords'].fillna('[]')
    creative_info['title_list'] = creative_info['title_list'].apply(json.loads)
    creative_info['ad_keywords'] = creative_info['ad_keywords'].apply(json.loads)

    now_plan_roi = get_now_plan_roi()
    now_plan_roi = pd.merge(now_plan_roi, creative_info, on=['channel_id', 'source_id'], how='left')

    df_create = pd.merge(plan_info, now_plan_roi, on=['channel_id', 'source_id'], how='inner')

    df_create['platform'] = df_create['platform'].astype(str)
    df_create['platform'] = df_create['platform'].map({"['ANDROID']": 1, "['IOS']": 2})
    df_create['platform'] = df_create['platform'].astype(int)

    df_create = df_create[df_create['platform'] == 1]
    df_create = df_create.dropna(subset=['ad_keywords','title_list','third_industry_id'],how='any')
    
    df_create['inventory_type'] = df_create['inventory_type'].astype(str)
    df_create['retargeting_tags_include'] = df_create['retargeting_tags_include'].astype(str)
    df_create['retargeting_tags_exclude'] = df_create['retargeting_tags_exclude'].astype(str)
    df_create['ad_keywords'] = df_create['ad_keywords'].astype(str)
    df_create['title_list'] = df_create['title_list'].astype(str)
    inventory_type_info = df_create['inventory_type'].value_counts().to_dict()
    retargeting_tags_include_info = df_create['retargeting_tags_include'].value_counts().to_dict()
    retargeting_tags_exclude_info = df_create['retargeting_tags_exclude'].value_counts().to_dict()
    ad_keywords_info = df_create['ad_keywords'].value_counts().to_dict()
    title_list_info = df_create['title_list'].value_counts().to_dict()
    third_industry_id_info = df_create['third_industry_id'].value_counts().to_dict()
    return inventory_type_info,retargeting_tags_include_info,retargeting_tags_exclude_info,ad_keywords_info,title_list_info,third_industry_id_info

In [58]:
# 获取image_id,label_ids
def get_image_info():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id,
            a.image_id,
            b.label_ids
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN
            db_data_ptom.ptom_image_info b
        on a.image_id = b.image_id
        WHERE
            a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
            AND a.create_time >= date( NOW() - INTERVAL 2880 HOUR ) 
        GROUP BY
            a.chl_user_id,
            a.source_id,
            a.image_id
    '''
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    result_df = result_df.dropna(axis=0, how='any')
    result_df['channel_id'] = result_df['channel_id'].map(int)
    result_df['source_id'] = result_df['source_id'].map(int)
    result_df['image_id'] = result_df['image_id'].map(int)
    cur.close()
    conn.close()
    return result_df

In [59]:
# 获取score_image (分数大于550的image_id)
def get_score_image():
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                    password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql_queue = 'set tez.queue.name=offline'
    sql = '''
        SELECT
            image_id,
            label_ids 
        FROM
            dws.dws_image_score_d 
        WHERE
            dt = CURRENT_DATE 
            AND media_id = 10 
            AND image_create_role_roi >= 0.005 
            AND ( image_create_role_retain_1d >= 0.07 OR image_create_role_retain_1d IS NULL ) 
            AND score >= 500 
        GROUP BY
            image_id,
            label_ids
    '''
    
    cursor.execute(sql_engine)
    cursor.execute(sql_queue)
    cursor.execute(sql)
    result = as_pandas(cursor)
    result['label_ids'] = result['label_ids'].astype(str)
    result['label_ids'] = result['label_ids'].apply(lambda x: x.strip('-1;') if '-1' in x else x)
    result['label_ids'] = pd.to_numeric(result['label_ids'],errors='coerce')
#     result = result[result['label_ids'].isin([27])]
#     print(result)
    # 关闭链接
    cursor.close()
    conn.close()
    
    return result['image_id'].values

In [60]:
def get_score_image_other():
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                    password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql_queue = 'set tez.queue.name=offline'
    sql = '''
        SELECT
            image_id,
            label_ids 
        FROM
            dws.dws_image_score_d 
        WHERE
            dt = CURRENT_DATE 
            AND media_id in (45,16,32) 
            AND image_create_role_roi >= 0.008 
            AND ( image_create_role_retain_1d >= 0.07 OR image_create_role_retain_1d IS NULL ) 
            AND score >= 550 
        GROUP BY
            image_id,
            label_ids
    '''
    cursor.execute(sql_engine)
    cursor.execute(sql_queue)
    cursor.execute(sql)
    result = as_pandas(cursor)
    result['label_ids'] = result['label_ids'].astype(str)
    result['label_ids'] = result['label_ids'].apply(lambda x: x.strip('-1;') if '-1' in x else x)
    result['label_ids'] = pd.to_numeric(result['label_ids'],errors='coerce')
#     result = result[result['label_ids'].isin([27])]
#     print(result)
    # 关闭链接
    cursor.close()
    conn.close()
    
    return result['image_id'].values

In [61]:
def filter_exclude_image(image_ids, media_id):
    image_ids = [str(i) for i in image_ids]
    image_ids = ','.join(image_ids)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            image_id,
            image_name
        FROM
            db_data_ptom.ptom_image_info
        WHERE
            image_id IN ({})
            AND NOT FIND_IN_SET({}, exclude_media_ids)
    '''
    result = pd.read_sql(sql.format(image_ids, media_id), conn)
    # result = result[result['image_name'].apply(lambda x:True if len(re.findall(r"QMZZ(.*?)-", x))>0 and int(re.findall(r"QMZZ(.*?)-", x)[0]) not in tt_ban else False)]
    # 关闭链接
    cur.close()
    conn.close()

    return result['image_id'].values

In [62]:
def filter_game_image(image_ids, game_id):
    image_ids = [str(i) for i in image_ids]
    image_ids = ','.join(image_ids)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            image_id,
            image_name
        FROM
            db_data_ptom.ptom_image_info
        WHERE
            image_id IN ({})
            AND FIND_IN_SET({}, game_ids)
    '''
    cur.execute(sql.format(image_ids, game_id))
    result = pd.read_sql(sql.format(image_ids, game_id), conn)

    # 关闭链接
    cur.close()
    conn.close()

    return result['image_id'].values

In [63]:
# 303机器人勿上
def get_lable_image():
    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
       SELECT
            image_id 
        FROM
            db_ptom.ptom_image_info a 
        WHERE
            FIND_IN_SET( 303, label_ids ) 

    '''
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    cur.close()
    conn.close()
    return result_df['image_id'].values

In [64]:
# 对长期累积roi不达标的素材进行过滤
# 获取数据-1-6天
def get_data_1_6():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                   password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql = '''
            SELECT
                user_id,
                game_id,
                channel_id,
                source_id,
                role_id,
                platform,
                media_id,
                pay_num,
                pay_sum,
                create_role_time,
                created_role_day,
                pay_7_pred,
                dt 
            FROM
                tmp_data.tmp_roles_portrait_info_predict 
            WHERE
                dt = CURRENT_DATE
                AND channel_id = 21206
                AND game_id IN ({})
    
    '''
    finalSql = sql.format(game_id)
    cursor.execute(finalSql)
    result = as_pandas(cursor)

    # 关闭链接
    cursor.close()
    conn.close()

    return result

def get_data_7():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                   password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql = '''
        SELECT
            user_id,
            game_id,
            channel_id,
            source_id,
            role_id,
            platform,
            media_id,
            pay_num,
            pay_sum,
            create_role_time,
            created_role_day,
            pay_sum AS pay_7_pred,
            dt 
        FROM
            tmp_data.tmp_roles_portrait_info_train2 
        WHERE
            dt = CURRENT_DATE 
            AND created_role_day = 7
            AND channel_id = 21206
            AND game_id IN ({})
    '''
    finalSql = sql.format(game_id)
    cursor.execute(finalSql)
    result = as_pandas(cursor)

    # 关闭链接
    cursor.close()
    conn.close()

    return result

# 获取近期所有计划的消耗情况
def get_amount_info():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
            SELECT
                tdate,
                channel_id,
                source_id,
                game_id,
                media_id,
                platform,
                amount 
            FROM
                db_stdata.st_lauch_report 
            WHERE
                game_id IN ({}) 
                AND tdate_type = 'day' 
                AND channel_id = 21206
                AND tdate >= date( NOW() - INTERVAL 7 DAY )
                AND tdate <= date( NOW() - INTERVAL 1 DAY )
    '''
    finalSql = sql.format(game_id)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    
    return result_df

def calculate_roi(groupby_list=['media_id','platform','image_id']):
    # 获取预测数据
    df_1 = get_data_7()
    df_2 = get_data_1_6()
    df = df_1.append(df_2)
    df['create_role_time'] = pd.to_datetime(df['create_role_time']).dt.date
    # 聚合生成计划维度的回款预测数据
    source_df = pd.DataFrame({'7_pay_sum':df.groupby(['game_id','channel_id','source_id','create_role_time','media_id','platform'])['pay_7_pred'].sum()}).reset_index()
    source_df['media_id'] = source_df['media_id'].map(int)
    source_df = source_df[source_df['media_id'].isin([10,16,32,45])]
    # 获取计划消耗数据，只取4个主流媒体
    amount_info = get_amount_info()
    amount_info = amount_info[amount_info['media_id'].isin([10,16,32,45])]
    image_info = get_image_info()
    # 拼接数据
    source_df.rename(columns={'create_role_time':'tdate'}, inplace=True)
    source_df['tdate'] = pd.to_datetime(source_df['tdate'])
    source_df = pd.merge(source_df,amount_info,on=['channel_id','source_id','tdate','media_id','platform','game_id'],how='outer')
    source_df['amount'] = source_df['amount'].fillna(0)
    source_df['7_pay_sum'] = source_df['7_pay_sum'].fillna(0)
    data = pd.merge(source_df,image_info,on=['channel_id','source_id'],how='left')
    
    df_pay_sum = pd.DataFrame({'7_pay_sum': data.groupby(groupby_list)['7_pay_sum'].sum()}).reset_index()
    df_amount = pd.DataFrame({'7_amount': data.groupby(groupby_list)['amount'].sum()}).reset_index()
    df = pd.merge(df_amount,df_pay_sum,on=groupby_list,how='left')
    df['7_pay_sum'] = df['7_pay_sum'].apply(lambda x: round(x,2))
    df['7_amount'] = df['7_amount'].apply(lambda x: round(x,2))
    df = df[df['7_amount']>0]
    df['7_roi'] = round(df['7_pay_sum'] / df['7_amount'],4)
    
    df = df[(df['media_id']==10)&(df['platform']==1)&(df['7_amount']>=15000)&(df['7_roi']<0.05)]
    df['image_id'] = df['image_id'].astype(int)
    return df['image_id'].values

In [65]:
# 历史拉环
def get_old_im():
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                   password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql = '''
      select distinct(image_id) from dws.dws_image_score_d where label_ids ='20;235' and dt >='2022-05-01' and score >=600
    '''
    cursor.execute(sql)
    result = as_pandas(cursor)

    # 关闭链接
    cursor.close()
    conn.close()

    return result['image_id'].values

In [66]:
def get_old_image():
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                    password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql_queue = 'set tez.queue.name=offline'
    sql = '''
        select image_id,image_name from dws.dws_image_score_d where dt>='2022-05-01' and image_name like '%SSR%' and score>=600 AND label_ids not in ('27', '279') group by image_id,image_name

    '''
    cursor.execute(sql_engine)
    cursor.execute(sql_queue)
    cursor.execute(sql)
    result = as_pandas(cursor)
    
#     result = result[result['label_ids'].isin([27])]
#     print(result)
    # 关闭链接
    cursor.close()
    conn.close()
    
    return result['image_id'].values

In [67]:
def select_image(game_id, media_id):
    # 选定高分素材，并对不能跑的素材进行过滤
    score_image_1 = get_score_image_other()
    score_image_2 = get_score_image()
    score_image = np.union1d(score_image_1,score_image_2)
#     # 跑历史拉环
#     score_image = get_old_im()
    score_image = filter_game_image(score_image, game_id)
    image_id_group = filter_exclude_image(score_image, media_id)
    image_id_group = np.setdiff1d(image_id_group,calculate_roi())
    image_id_group = np.setdiff1d(image_id_group, get_lable_image())  # 过滤指定标签的素材
    return image_id_group

In [68]:
def select_image_old(game_id, media_id):
    # 选定高分素材，并对不能跑的素材进行过滤
    score_image = get_old_image()
    score_image = filter_game_image(score_image, game_id)
    image_id_group = filter_exclude_image(score_image, media_id)
    image_id_group = np.setdiff1d(image_id_group,calculate_roi())
    image_id_group = np.setdiff1d(image_id_group, get_lable_image())  # 过滤指定标签的素材
    return image_id_group

In [69]:
def get_ad_create(plan_result):
    ad_info = []
    for i in range(plan_result.shape[0]):
        ad_info.append(json.loads(plan_result.iloc[i].to_json()))
    open_api_url_prefix = "https://ptom.caohua.com/"
#     open_api_url_prefix = "https://ptom-pre.caohua.com/"   ## 预发布环境
    uri = "model/generationPlanBatchTask"
    url = open_api_url_prefix + uri
    params = {
        "secretkey": "abc2018!@**@888",
        "mediaId": 10
    }
    rsp = requests.post(url, json=ad_info, params=params)
    rsp_data = rsp.json()
    print('结束....')
    return rsp_data

In [70]:
# image_id_group = select_image(global_var.game_id, global_var.media_id)
# print('匹配到的素材', image_id_group)

In [71]:
image_id_group = select_image(global_var.game_id, global_var.media_id)
if image_id_group.size <= 2 * global_var.plan_num:
    image_id_group = np.union1d(image_id_group,np.random.choice(select_image_old(global_var.game_id, global_var.media_id),4 * global_var.plan_num))
    
print('匹配到的素材', image_id_group)

匹配到的素材 [42241 44524 45415 45534 45795 45799 46109 47425 47539 47898 48003 48225
 48243 48256]


In [72]:
inventory_type_info,retargeting_tags_include_info,retargeting_tags_exclude_info,ad_keywords_info,title_list_info,third_industry_id_info = get_info()

In [73]:
plan_result = pd.DataFrame()
for ad_account in global_var.ad_account_id_group:
    image_id_temp = np.random.choice(image_id_group,global_var.plan_num,replace=False)
    for image_id in image_id_temp:
        temp = pd.DataFrame({'ad_account_id': [ad_account], 'image_id': [image_id]})
        plan_result = plan_result.append(temp)

In [74]:
# 基础固定参数
plan_result['game_id'] = global_var.game_id
plan_result['platform'] = global_var.platform
plan_result['budget'] = global_var.budget
plan_result['android_osv'] = "None"
plan_result['ios_osv'] = "None"
plan_result['inventory_type'] = plan_result.apply(lambda x:np.random.choice(list(inventory_type_info.keys()), p=np.divide(list(inventory_type_info.values()), sum(list(inventory_type_info.values()))).tolist()), axis=1)
plan_result['delivery_range'] = plan_result.apply(lambda x:'UNION' if x.inventory_type == "['INVENTORY_UNION_SLOT']" else 'DEFAULT', axis=1)
plan_result['schedule_time'] = plan_result['ad_account_id'].apply(lambda x: "111111111111111111111111111111111111111111"
                                                                            "111111111111111111111111111111111111111111"
                                                                            "111111111111111111111100000000001111111111"
                                                                            "111111111111111111111111111100000000001111"
                                                                            "111111111111111111111111111111111111111111"
                                                                            "111111111111111111111111111111111111111111"
                                                                            "111111111111111111111111111111111111111111"
                                                                            "111111111111111111111111111111111111111111")
plan_result['flow_control_mode'] = "FLOW_CONTROL_MODE_FAST"                                        #竞价策略
plan_result['plan_auto_task_id'] = "11002,10998,12098"
plan_result['op_id'] = 13268
plan_result['district'] = 'CITY'
plan_result['flag'] = 'SSR'
plan_result['web_url'] = "https://www.chengzijianzhan.com/tetris/page/7127204719681159175/"


In [75]:
# 定向池  retargeting_tags_include\retargeting_tags_exclude\
# plan_result['retargeting_tags_include'] = plan_result.apply(lambda x:np.random.choice(list(retargeting_tags_include_info.keys()), p=np.divide(list(retargeting_tags_include_info.values()), sum(list(retargeting_tags_include_info.values()))).tolist()), axis=1)
# plan_result['retargeting_tags_exclude'] = plan_result.apply(lambda x:np.random.choice(list(retargeting_tags_exclude_info.keys()), p=np.divide(list(retargeting_tags_exclude_info.values()), sum(list(retargeting_tags_exclude_info.values()))).tolist()), axis=1)
# plan_result['retargeting_tags_include'] = "[]"
# plan_result['retargeting_tags_exclude'] = "[341029460, 341029326]"
plan_result['retargeting_tags_include'] = "[]"
plan_result['retargeting_tags_exclude'] = "[]"

plan_result['city'] = plan_result.apply(lambda x:"[]", axis=1)
plan_result['location_type'] = "CURRENT"
plan_result['gender'] = plan_result.apply(lambda x:np.random.choice(["GENDER_MALE","None"],p=[0.7,0.3]), axis=1)
plan_result['age'] = plan_result.apply(lambda x:"['AGE_BETWEEN_24_30', 'AGE_BETWEEN_31_40', 'AGE_BETWEEN_41_49']", axis=1)
plan_result['ac'] = "[]"
plan_result['launch_price'] = plan_result.apply(lambda x:np.random.choice(["[]", "[2000, 11000]"],p=[0.9,0.1]), axis=1)
plan_result['auto_extend_enabled'] = 0
plan_result['hide_if_exists'] = 0
plan_result['hide_if_converted'] = plan_result.apply(lambda x:np.random.choice(["AD", "CUSTOMER"],p=[0.65,0.35]), axis=1)

# 兴趣
plan_result['interest_action_mode'] = 'UNLIMITED'
plan_result['action_scene'] = np.nan
plan_result['action_days'] = np.nan
plan_result['action_categories'] = np.nan
plan_result['interest_categories'] = np.nan



In [76]:
# 创意参数
plan_result['ad_keywords'] = plan_result.apply(lambda x:np.random.choice(list(ad_keywords_info.keys()), p=np.divide(list(ad_keywords_info.values()), sum(list(ad_keywords_info.values()))).tolist()), axis=1)
plan_result['title_list'] = plan_result.apply(lambda x:np.random.choice(list(title_list_info.keys()), p=np.divide(list(title_list_info.values()), sum(list(title_list_info.values()))).tolist()), axis=1)
plan_result['third_industry_id'] = plan_result.apply(lambda x:np.random.choice(list(third_industry_id_info.keys()), p=np.divide(list(third_industry_id_info.values()), sum(list(third_industry_id_info.values()))).tolist()), axis=1)

In [77]:
# 出价
plan_result['cpa_bid'] = plan_result.apply(lambda x:random.randint(760, 800), axis=1)
plan_result['smart_bid_type'] = plan_result.apply(lambda x:np.random.choice(['SMART_BID_CUSTOM', 'SMART_BID_NO_BID']), axis=1)
plan_result['deep_bid_type'] = 'BID_PER_ACTION'
plan_result['roi_goal'] = np.nan
plan_result['adjust_cpa'] = 0
plan_result['convertIndex'] = plan_result['deep_bid_type'].apply(lambda x:23 if x == 'BID_PER_ACTION'
                                                                                else 24 if x == 'ROI_COEFFICIENT'
                                                                                    else np.nan)

In [78]:
# 穿山甲版位不支持nobid
plan_result['inventory_type'] = plan_result['inventory_type'].apply(lambda x: "[]" if x ==
                  "['INVENTORY_UNION_SLOT', 'INVENTORY_AWEME_FEED', 'INVENTORY_FEED', 'INVENTORY_UNION_SPLASH_SLOT', "
                  "'INVENTORY_VIDEO_FEED', 'INVENTORY_HOTSOON_FEED', 'INVENTORY_TOMATO_NOVEL']" else x)
plan_result['smart_bid_type'] = plan_result.apply(lambda x: 'SMART_BID_CUSTOM' if 'INVENTORY_UNION_SLOT' in x.inventory_type else x.smart_bid_type, axis=1)

In [79]:
plan_result['inventory_type'] = plan_result['inventory_type'].apply(ast.literal_eval)
plan_result['platform'] = plan_result['platform'].apply(ast.literal_eval)
plan_result['retargeting_tags_include'] = plan_result['retargeting_tags_include'].apply(ast.literal_eval)
plan_result['retargeting_tags_exclude'] = plan_result['retargeting_tags_exclude'].apply(ast.literal_eval)
plan_result['city'] = plan_result['city'].apply(ast.literal_eval)
plan_result['age'] = plan_result['age'].apply(ast.literal_eval)
plan_result['ac'] = plan_result['ac'].apply(ast.literal_eval)
plan_result['launch_price'] = plan_result['launch_price'].apply(ast.literal_eval)
plan_result['ad_keywords'] = plan_result['ad_keywords'].apply(ast.literal_eval)
plan_result['title_list'] = plan_result['title_list'].apply(ast.literal_eval)
# 限制手机价格
plan_result['launch_price'] = plan_result['launch_price'].apply(lambda x: "[3000, 11000]")
plan_result['launch_price'] = plan_result['launch_price'].apply(ast.literal_eval)
# plan_result['operation'] = 'disable'

In [80]:
print('计划数量%d' % plan_result.shape[0])
rsp_data = get_ad_create(plan_result)
print(rsp_data)

计划数量15
结束....
{'code': 200, 'data': True, 'msg': ''}


In [81]:
plan_result.to_csv('./plan_result.csv',index=0)

In [82]:
plan_result['image_id'].value_counts()

46109    3
48225    2
48003    2
47539    2
47425    2
45534    2
45799    1
45795    1
Name: image_id, dtype: int64